In [0]:
from pyspark.sql.functions import col

# Lire les tables Silver
crm_cust_df = spark.table("lakehouse.silver.crm_cust_info")
erp_cust_df = spark.table("lakehouse.silver.erp_custt")
erp_loc_df = spark.table("lakehouse.silver.erp_locationn")

# Jointure ERP cust + location
erp_full_df = erp_cust_df.join(
    erp_loc_df,
    erp_cust_df["CUST_ID"] == erp_loc_df["CUST_ID"],
    how="left"
)

# Jointure CRM cust info + ERP full
gold_dim_cust_df = crm_cust_df.join(
    erp_full_df,
    crm_cust_df["cst_key"] == erp_full_df["CUST_ID"],
    how="left"
)

# Sélection finale Gold
gold_dim_cust_df = gold_dim_cust_df.select(
    col("cst_id"),
    col("Customer_id"),
    col("CUST_ID"),
    col("Date_birth"),
    col("Customer_Gender").alias("customer_gender"),
    col("Country"),
    col("cst_firstname"),
    col("cst_lastname"),
    col("cst_marital_status"),
    col("cst_gndr"),
    col("cst_create_date")
)

# Écriture Gold layer
gold_dim_cust_df.write.mode("overwrite").format("delta").saveAsTable("lakehouse.gold.dim_customerss")

print("Gold dimension dim_customerss created successfully")
